In [1]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

from IPython.display import display, Image

# Introduction to Shortest Path and Pathfinding
### Designed by Eric Ianni

### Pre-requisits
* Knowledge of how to use a Jupyter Notebook
* Knowledge of basic Python and the use of multi-dimensional arrays
* Knowledge of basic weighted graphs

### Let's Paint a Picture!

Imagine you are a playing a turn-based computer game where you have a limited number of movement points per turn. Your task is to start at the village inthe top left and make it to the chest in the castle as quickly as possible without moving diagnolly.

![Castle image](./images/castle_map.png)

To you, as a human, it is very easy to see that the only clear path is around the North of the castle, back down the East and across the bridges in the South.  You are able to process the entire map at once and quickly identify dead ends.

To a computer this task isn't that easy. Computers are limited in how much of the "big picture" they can see and have to approach problems in single steps at a time. 

Now imagine you are told that it is possible to cross water tiles, but there is a movement penalty. All of a sudden the task isn't so easy as you have to consider if swimming the moat is faster than the long way. Still, you can probably figure out that crossing the one water tile to the South-West is probably the fastest route.

Now imagine it is possible to climb the castle walls, but with a stiff movement penalty. Can you still confidently say for sure you know the fastest path without sitting down and doing the math for multiple different routes?

In this latest scenario you find yourself approaching the problem how a computer  does!

### Welcome to Pathfinding!

In this module you will learn about how computers solve the _Shortest Path Problem_ by using different _Pathfinding_ algorithms. We will begin by examining real world problems that require a shortest path. Next, we will learn to frame those problems as graphs so we can apply different algorithms. Finally, we will examine these algorithms and how each approach the problem differently. 

### Learning Objectives

* Define the _Shortest Path Problem_ and give examples
* Describe _Depth First Search_ and _Breadth First Search_ while citing the differences and the required data structures
* Describe Dijkstra's algorithm and implement
* Describe the A* algorithm and implement

## What is Pathfinding?
The simplest definition of _pathfinding_ is finding a route (_path_) between points *A* and *B*. We do this all the time everyday of our lives. When driving to the store we have to pick which streets to follow to get there. Even the simple act of crossing a room is an act of selecting a route, even more so if you have children leaving toys on the ground!

While you or I use our eyes and intuition for many _pathfinding_ tasks, computers must rely on other approaches. Most computational _pathfinding_ makes heavy use of _graphy theory_. This involves defining the given situation as a _graph_: a series of nodes and edges. 

To demonstrate this process, let's imagine a bus-network between a few cities: New York City, Boston, Washington D.C., Columbus, and Chicago. To represent the network, we begin by designating each city a _vertex_ in the graph. We then create an _edge_ between vertices to mirror the bus-routes. Finally we can assign the number of miles between the cities as the edge _weights_.

![City Graph](./images/city_graph.png)

Now that we have learned how to frame a problem in a way for computers to understand, it is time to actually find a path.

## Introducing Twiddle Dee and Twiddle Dumb

The first two algorithms we are going to examine are _Depth First Search_ (DFS) and _Breadth First Search_ (BFS). These two methods are what I would call _dumb_ in that they are brute force approaches to the problem of _pathfinding_. 

They get their name from how they search through _tree graphs_. DFS searches down a single branch until it reaches a leaf before it backtracks. BFS, on the other hand, searches each child node before moving onto lower levels of the tree.  

![DFS vs. BFS in Binary Trees](./images/binary_tree_search.png)(image borrowed from http://mishadoff.com/blog/dfs-on-binary-tree-array/)

When not being used for _pathfinding_, DFS and BFS return a list of nodes in the order in which they were visited/searched. Given the binary-trees above:

* DFS would return **[0,1,3,4,2,5,6]** 
* BFS would return **[0,1,2,3,4,5,6]**

## Let's see these two in action!

For this section we are going to be using the following graph.

![DFS & BFS Graph](./images/dfs_bfs_graph.png)
(Image borrowed from https://eddmann.com/posts/depth-first-search-and-breadth-first-search-in-python/)

Below you will find two code snippets that implement DFS and BFS. You will be able to run these algorithms with different parameters to get a feel for how each search works. 

Begin by simply running the cells (ctrl-enter) and trying different values for the starting node and the goal node. Try the same values for start and goal for each algorithm. 

After you have experimented, try to answer the following questions:

1. Why don't DFS and BFS come up with the same path?
2. Why does BFS seem to be able to find shorter paths than DFS? (look at start: 'A' and goal: 'C')
3. Pay attention to the number of steps required, why are they different?
4. What does that mean for the value for each of these searches?

Now go have fun!

### Depth First Search Playground

In [7]:
# Code modified from https://eddmann.com/posts/depth-first-search-and-breadth-first-search-in-python/

graph = {'A': set(['B', 'C']),
         'B': set(['A', 'D', 'E']),
         'C': set(['A', 'F']),
         'D': set(['B']),
         'E': set(['B', 'F']),
         'F': set(['C', 'E'])}
            
def dfs(graph, start, goal):
    steps = 0
    stack = [(start, [start])]
    while stack:
        steps += 1
        (vertex, path) = stack.pop()
        if vertex == goal:
            return path, steps
        for next in graph[vertex] - set(path):
            stack.append((next, path + [next]))

def run_dfs(start, goal):
    path, steps = dfs(graph, start, goal)
    print("Path: " + str(path))
    print("Steps: " + str(steps))

interact(run_dfs, start=['A','B','C','D','E','F'], goal=['A','B','C','D','E','F'])


interactive(children=(Dropdown(description=u'start', options=('A', 'B', 'C', 'D', 'E', 'F'), value='A'), Dropdown(description=u'goal', options=('A', 'B', 'C', 'D', 'E', 'F'), value='A'), Output()), _dom_classes=('widget-interact',))

<function __main__.run_dfs>

### Breath First Search Playground

In [6]:
# Code modified from https://eddmann.com/posts/depth-first-search-and-breadth-first-search-in-python/

graph = {'A': set(['B', 'C']),
         'B': set(['A', 'D', 'E']),
         'C': set(['A', 'F']),
         'D': set(['B']),
         'E': set(['B', 'F']),
         'F': set(['C', 'E'])}

def bfs(graph, start, goal):
    steps = 0
    queue = [(start, [start])]
    while queue:
        steps += 1
        (vertex, path) = queue.pop(0)
        if vertex == goal:
            return path, steps
        for next in graph[vertex] - set(path):
            queue.append((next, path + [next]))

def run_bfs(start, goal):
    path, steps = bfs(graph, start, goal)
    print("Path: " + str(path))
    print("Steps: " + str(steps))

interact(run_bfs, start=['A','B','C','D','E','F'], goal=['A','B','C','D','E','F'])


interactive(children=(Dropdown(description=u'start', options=('A', 'B', 'C', 'D', 'E', 'F'), value='A'), Dropdown(description=u'goal', options=('A', 'B', 'C', 'D', 'E', 'F'), value='A'), Output()), _dom_classes=('widget-interact',))

<function __main__.run_bfs>

### Time to Debrief

I hope you had fun experimenting with both searches. Did you have time to answer those questions?

1. Why don't DFS and BFS come up with the same path?
   * DFS rushes head long down a single path until it hits a dead end. BFS cautiously expands the search radius little by little
2. Why does BFS seem to be able to find shorter paths than DFS? (look at start: 'A' and goal: 'C')
   * BFS's cautious approach means if the goal node is close by, it doesn't run past it like DFS does
3. Pay attention to the number of steps required, why are they different?
   * If DFS gets lucky and picks the correct path to pursue, it will find the goal with fewer steps, even if it is far away. BFS is very efficient when finding closer goals, but if the graph has many branches and the goal is far away, the number of required steps escalates quickly!
4. What does that mean for the value for each of these searches?
   * With BFS you are guaranteed the shortest path but you may have to examine many more nodes. With DFS, you are guaranteed to find _a_ path and there is no requirement to completely cover the graph, but the path isn't guaranteed to be the shortest. What search to use depends on the number of nodes to search and how important it is to have the shortest path

   
This leads nicely into our next topic!

## What is a Shortest Path Problem?
Simply put, it is any problem that requires finding the path between two points that is deemed the _shortest_. In the context of _graphs_, one path is shorter than another path when the sum of the _weights_ is smaller. Both _Directed_ and _Undirected_ graphs can be used to represent _shortest path problems_.

Let's look at an example. Below is a weighted graph and our task is to find the _shortest path_ from *A* to *F*.

![Weighted Graph 1](./images/graph1.png) 

At first glance it may look like _A->E->F_ is the answer as it only has two steps, but it has a total weight of 12.

![Weighted Graph 1 AEF](./images/graph1AEF.png)

The actual _shortest path_ is A->B->D->C->E->F. While it takes a whopping 5 steps(!), it has a total weight of 11.

![Weighted Graph 1 ABDCEF](./images/graph1ABDCEF.png)

## Shortest Paths in the Wild
There are countless examples of real life problems that require knowing the _shortest path_. 

* Navigating with Google Maps
* Routing internet traffic
* Enemy movement in a video game 

As long as the problem can be represented as a weighted graph, it can be a _shortest path problem_. In the case of driving directions, intersections can be used as nodes while the driving time between nodes can be used as the edge weight. This graph can even be manipulated in real time to represent traffic delays so that the _shortest path_ changes.

## Shortest Pathfinding Algorithms

Now we know what a _Shortest Path Problem_ is and we already learned about an algorithm, BFS, that is guaranteed to find the shortest path. So we are done right? Let's go home!

Sadly, BFS has some pretty big issues when it comes to pathfinding.

* It _only_ works on non-weighted graphs. This means it will never be able to take into account things like distance between towns when mapping out directions
* It has to search the every branch it comes acrosss before it can decide on a path. Imagine Google Maps spending time investigating roads that go in the wrong direction before it finds the correct path

But there is hope! There are dozens of approaches that smarter folks than me have devised to solve these types of problems. We are going to look at just two of them in this module.

1. Dijkstra's Algorithm
2. A* Algorithm


### Dijkstra's

As I am sure you guessed, Dijkstra's Algorithm was invented by a gentleman named Edsger Dijkstra in 1956. He claims to have invented it at a cafe in Amsterdam in the span of 20 minutes.

The basic algorithm is:

1. Add the source node to a priority queue with a priority of 0
2. Select the node from the queue with the highest priority
3. Select the neighbor of the current node with the lowest cost to visit. To calculate this, add the cost to reach the current node from the source vertex to the cost of traveling to the neighbor
4. If this total cost is less than the known cost to reach each neighbor node, update the cost and add the neighbor to the queue with a priority of the new cost
5. Repeat steps 2-4 until you find the target node

These plain English steps can be formalized into the properly formatted listed below. Please note that the algorithm below has a data structure called _prev_ to help with building a path once the search is complete.

```python
def dijkstra(graph, source, target):
    create empty Priority Queue Q
    Put source into Q with priority 0
    for each vertex v in graph:
        cur_costs[v] <- INF
        prev[v] <- NULL
    while Q is not empty:
            current <- get highest priority vertex
            if current == target:
                break
            for each neighbor n of current:
                new_cost <- cur_costs[current] + graph.costs[n]
                if new_cost < cur_costs[n]:
                    cur_costs[n] <- new_cost
                    add n to Q with priority new_cost
                    prev[n] <- current
    return cur_costs, prev
```

Armed with this information I believe we are ready to try our hand at implementing Dijkstra's Algorithm on a some small maps similar to our castle example.

### Let's Do This!
For the remainder of this module we are going to use graphs that are represented by grids. This will help us visualize pathfinding a bit easier. It also makes the code more straightforward. We will be storing these maps in 2D Python lists that store different terrain types as _chars_. Below you will find a sample map along with a visual representation to make it easier to see what is going on.

```python
'''
s = source
t = target
g = grass
f = forest
h = hill
m = mountain
w = wall
'''

map_1 = [['s','g','g','f','f','g','g','f'],
         ['g','g','f','h','h','f','g','g'],
         ['f','f','h','m','h','f','h','g'],
         ['f','h','m','h','f','h','m','h'],
         ['f','h','h','f','g','g','h','m'],
         ['f','f','f','g','g','g','g','h'],
         ['g','g','g','g','h','g','g','g'],
         ['g','m','m','m','m','h','g','t']]

``` 
<img src="./images/small_map1.png" align="left"/>
<div style="clear: both;"></div>

So now that we have a way to represent our graphs as a map we need a way to convert this into something usable by our algoirthm. So let's make a Graph _class_! What does this class need to do?

1. Parse map of chars into cost of movement into each grid square
2. Know map specifics such as height, width, source, and target
3. Provide access to this information

We will first design our map parser. It will take in a 2D Python list like the one above and convert all those _chars_ into costs of movement. It will also identify which gridsquares are the source and target.

```python
# Accepts a map of chars as grid
def parse_map(self, grid):
        # Save dimensions
        self.height = len(grid)
        self.width = len(grid[0])
        
        # Set source and target in case the map doesn't specify
        self.source = [0,0]
        self.target = [len(grid)-1, len(grid[0])-1]
    
        # Build an array of same dimensions to store costs of movement (default 1)
        self.costs = [[1 for col in range(len(grid[row]))] for row in range(len(grid))]
    
        # Iterate over map of chars
        for row in range(len(self.costs)):
            for col in range(len(self.costs[0])):
                # Found the source!
                if grid[row][col] == 's':
                    self.source = [row,col]
                # Found the target!
                elif grid[row][col] =='t':
                    self.target = [row,col]
                # Look up the cost for terrain type and assign to costs grid
                else:
                    self.costs[row][col] = self.cost_legend[grid[row][col]]
```

The rest of the functionality is straightforward so see the complete _Graph_ class below.

In [8]:
class Graph:
    
    # Cost for moving into each terrain type
    cost_legend = {'g':1, 'f':2, 'h':3, 'm':4, 'w':float('inf')}
    
    def __init__(self, grid):
        self.map = grid
        self.parse_map(grid)
        
    # Accepts a map of chars as grid
    def parse_map(self, grid):
        # Save dimensions
        self.height = len(grid)
        self.width = len(grid[0])
        
        # Set source and target in case the map doesn't specify
        self.source = [0,0]
        self.target = [len(grid)-1, len(grid[0])-1]
    
        # Build an array of same dimensions to store costs of movement (default 1)
        self.costs = [[1 for col in range(len(grid[row]))] for row in range(len(grid))]
    
        # Iterate over map of chars
        for row in range(len(self.costs)):
            for col in range(len(self.costs[0])):
                # Found the source!
                if grid[row][col] == 's':
                    self.source = [row,col]
                # Found the target!
                elif grid[row][col] =='t':
                    self.target = [row,col]
                # Look up the cost for terrain type and assign to costs grid
                else:
                    self.costs[row][col] = self.cost_legend[grid[row][col]]
    
    # Returns true if the passed loc is within the grid
    def in_bounds(self, loc):
        row, col = loc
        return 0 <= row < self.height and 0 <= col < self.width
    
    # Returns the cost of moving into the gridsquare loc
    def cost(self, loc):
        row, col = loc
        return self.costs[row][col]

### Priority Queue
Now that we can represent our map as a graph, we need to have a way for Dijkstra's algorithm to keep track of our ever expanding list of nodes that need processing. For this we need a _Priority Queue_. While there are ready-made classes available we don't need all those features and they are easy to implement. For our purposes the _Priority Queue_ needs the following features:

1. Store all the elements
2. Add elements to queue
3. Sort elements so the item with the highest priority is at the front
4. Remove the first item and return it

For our implementation we will use a simple List which we will call *Q* (get it?). When adding elements to the queue we will simply append them to the list. To ensure that the highest priority element is on the top we will use the built in list _sort()_ function. This sorts from highest to lowest priority so when using retrieving the highest priority element we need to use _pop(0)_. The _empty()_ function will just check the length of the queue.

Please see the full class below.

In [9]:
class Priority_Queue():
    
    # Declare list to use for the queue
    def __init__(self):
        self.Q = []
    
    # Add item to queue and sort by priority
    def put(self, item):
        self.Q.append(item)
        self.Q.sort()
    
    # Pop off the front of the queue and return item
    def get(self):
        if not self.empty():
            item = self.Q.pop(0)
            return item
        else:
            return None
    # Returns true if the queue is empty
    def empty(self):
        return len(self.Q) == 0

### Some Utility Code

The code below will be required when we finally implement our search algorithms. The first part consists of our map data. This is followed by two lists _directions_, which will help us quickly find the neighbors for each gridsquare, and <i>direction_symbols</i>, which will be used for visualizing the path once we found it. Finally, we have <i>print_grid</i> which makes it easy to display any sort of 2D list, whether it is a map, cost grid, or even the final path.

In [10]:
map_1 = [['s','g','g','f','f','g','g','f'],
         ['g','g','f','h','h','f','g','g'],
         ['f','f','h','m','h','f','h','g'],
         ['f','h','m','h','f','h','m','h'],
         ['f','h','h','f','g','g','h','m'],
         ['f','f','f','g','g','g','g','h'],
         ['g','g','g','g','h','g','g','g'],
         ['g','m','m','m','m','h','g','t']]

map_2 = [['s','g','g','f','f','g','g','f'],
         ['g','g','f','h','h','f','g','g'],
         ['f','f','h','m','h','f','h','g'],
         ['f','h','m','h','f','h','m','h'],
         ['f','h','h','f','g','f','h','m'],
         ['f','f','f','g','g','g','g','h'],
         ['g','g','g','g','h','g','g','g'],
         ['g','m','m','m','m','h','g','t']]

map_3 = [['s','g','g','g','g','g','g','g'],
         ['g','g','g','g','g','g','g','g'],
         ['g','g','g','g','g','g','g','g'],
         ['g','g','g','g','g','g','g','g'],
         ['g','g','g','g','g','g','g','g'],
         ['g','g','g','g','g','g','g','g'],
         ['g','g','g','g','g','g','g','g'],
         ['g','g','g','g','g','g','g','t'],
        ]

directions = [[-1, 0 ], # Up
             [ 0, -1],  # Left
             [ 1, 0 ],  # Down
             [ 0, 1 ]]  # Right

direction_symbols = ["^", # Up 
                     "<", # Left
                     "V", # Down
                     ">"] # Right

def print_grid(name, grid):
    print(name + ": ")
    for row in grid:
        print(row)
    print()

### Writing Dijkstra's Algorithm

Here will be video showing the actual writing of the code the follows. The video will feature voice-overs that describe what each portion of the algorithm does and the design motivations.

In [ ]:
def dijkstra(graph):
    Q = Priority_Queue()
    Q.put([0, graph.source])
    cur_costs = [[float('inf') for col in range(graph.width)] for row in range(graph.height)]
    prev = [[' ' for col in range(graph.width)] for row in range(graph.height)]
    cur_costs[graph.source[0]][graph.source[1]] = 0
    steps = 0
    
    while not Q.empty():
        steps += 1
        # Get the closest node
        closest = Q.get()
        current = closest[1]
        
        if current == graph.target:
            return cur_costs, prev, steps

        for i in range(len(directions)):
            n_row = current[0] + directions[i][0]
            n_col = current[1] + directions[i][1]
            if graph.in_bounds([n_row, n_col]):
                new_cost = cur_costs[current[0]][current[1]] + graph.cost([n_row, n_col])
                if new_cost < cur_costs[n_row][n_col]:
                    cur_costs[n_row][n_col] = new_cost
                    Q.put([new_cost, [n_row, n_col]])
                    prev[n_row][n_col] = i
                    
    return cur_costs, prev, steps

### One Last Thing!
So we now have a complete search algorithm! The last step is to visualize the path from the _source_ to the _target_. We will need one more function <i>build_path</i>. Remember that in the search algorithm we produced a 2D list called _prev_ in which we stored the direction that was used to move into each gridsquare. Our function, <i>build_path</i>, starts at the _target_ and works backwards, updating the _path_ grid with the symbols corresponding to the the stored directions in _prev_. It does this until it reaches the _source_ node.

In [19]:
# Builds path by working backwards from target
def build_path(prev, source, target):
    # Create a 2D list of the same size as the map
    path = [[' ' for col in range(len(prev[row]))] for row in range(len(prev))]
    
    # Get target node
    row, col = target
    
    # Designate the target node in the path grid
    path[row][col] = '*'
    
    # Continue until reaching the source node
    while [row, col] != source:
        # Apply the stored movement direction to the current node to find next node
        new_row = row - directions[prev[row][col]][0]
        new_col = col - directions[prev[row][col]][1]
        # Insert the corresponding direction symbol to the next node
        path[new_row][new_col] = direction_symbols[prev[row][col]]
        # Update the current node
        row = new_row
        col = new_col
    return path

In [18]:
maps = [map_1, map_2, map_3]
map_files = ['small_map1.png', 'small_map2.png', 'small_map3.png']

def run_dijkstra(map_select):
    graph = Graph(maps[map_select - 1])
    costs, prev, steps = dijkstra(graph)
    print_grid("Costs:", costs)
    path = build_path(prev, graph.source, graph.target)
    print_grid("Path", path),
    print("Number of steps: " + str(steps))
    display(Image(filename='./images/' + map_files[map_select-1]))
    
interact(run_dijkstra, map_select=[1,2,3])

interactive(children=(Dropdown(description=u'map_select', options=(1, 2, 3), value=1), Output()), _dom_classes=('widget-interact',))

<function __main__.run_dijkstra>